In [0]:
# %pip install "typing_extensions==4.5.0" "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" "langchain"

LangChain is built to customize any LLM (e.g., GPT-4, Dolly2) with your own data, allowing the model to be fine-tuned on any custom dataset

Models LLMs: (20+ Integrations), Chat Models and Text Embedding Models (10+ Integrations)

Prompts: The style of treating inputs to pass into the model

Parsers: Take the outputs of the model and parse them into a more structured format.

# Imports

In [0]:
import torch
from transformers import pipeline

# langchain prompts
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# langchain parsers
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Models

## Define Model Architecture

- Dolly: https://huggingface.co/databricks

In [0]:
architecture = "databricks/dolly-v2-7b"

## Define Pipeline

In [0]:
dolly = pipeline(
    model=architecture,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    return_full_text=True,
)

In [0]:
def get_response(prompt):
    response = dolly(prompt)
    return response[0]["generated_text"]

In [0]:
response = get_response("What is 1+1?")
print(response)

What is 1+1?
In mathematics, 1+1 is a form of words meaning two.

It is used in two main situations.

When giving an instruction to perform a task, the syntax is "1+1": perform the task, such as sorting something.

When asking a question, the syntax is "1+1": Q: What is 2? A: 2 is a number. Q: What is 3? A: 3 is a number. Q: What is 4? A: 4 is a number. Q: What is 5? A: 5 is a number. Q: What is 6? A: 6 is a number. Q: What is 7? A: 7 is a number. Q: What is 8? A: 8 is a number. Q: What is 9? A: 9 is a number. Once you have asked 9 questions, you need to perform 1+1, which in this case is a task of sorting the numbers you have been given, so 7 + 8 + 9, which is what is called a triad, is 15, then you can say 9 + 8 + 7 is what is called a quads, which is 30.


# Prompts

## EMail Example

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [0]:
customer_email_style = """
American English \
in a calm and respectful tone.
"""

### Without Langchain

In [0]:
prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {customer_email_style}\
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks into a style that is 
American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [0]:
response = get_response(prompt)
print(response)

Translate the text that is delimited by triple backticks into a style that is 
American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```

Hello friend. I appreciate you bringing this to my attention. 
Unfortunately, I agree that this blender is having a bit of an issue and I am going to need to ask you to surrender it to the store manager for a replacement. 
I will take the blender, but you will be responsible for the cost of any damage you may have caused with the smoothie incident. I hope you understand.

-Dave


### With Langchain

In [0]:
prompt_template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {email_style}\
text: ```{email}```
"""

In [0]:
prompt_with_context = PromptTemplate.from_template(prompt_template_string)
print(prompt_with_context.input_variables)
print(prompt_with_context.template)

['email', 'email_style']
Translate the text that is delimited by triple backticks into a style that is {email_style}text: ```{email}```



In [0]:
hf_pipeline = HuggingFacePipeline(pipeline=dolly)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [0]:
llm_context_chain

Out[14]: LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['email', 'email_style'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {email_style}text: ```{email}```\n', template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers_modules.databricks.dolly-v2-7b.97611f20f95e1d8c1e914b85da55cc3937c31192.instruct_pipeline.InstructionTextGenerationPipeline object at 0x7f9d7a87bcd0>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None), output_key='text')

In [0]:
print(customer_email)


Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!



In [0]:
# Call the LLM to translate to the style of the customer message
print(llm_context_chain.predict(email=customer_email, email_style=customer_email_style))


Hi! My blender lid flew off and now my kitchen is covered in smoothie! I need your help to get it cleaned up!

Would you be able to help me? Please let me know by clicking the "agree" button below!


In [0]:
service_email = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [0]:
service_email_style = """\
a polite tone \
that speaks in English Pirate\
"""

In [0]:
# Call the LLM to translate to the style of the customer message
print(llm_context_chain.predict(email=service_email, email_style=service_email_style))


Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!


# Parsers

## Review Example

In [0]:
# Let's start with defining how we would like the LLM output to look like:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

Out[20]: {'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [0]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

### Without Parser

In [0]:
prompt_with_context = PromptTemplate.from_template(review_template)
print(prompt_with_context.input_variables)
print(prompt_with_context.template)

llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)
print(llm_context_chain)

['text']
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}

memory=None callbacks=None callback_manager=None verbose=False prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,

In [0]:
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [0]:
# Call the LLM to translate to the style of the customer message
response = llm_context_chain.predict(text=customer_review)
print(response)


True,False,2,amazon,47,60,40,50,99,39,20


### With Parser

In [0]:
gift_schema = ResponseSchema(
    name="gift",
    description="""Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.""",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="""How many days did it take for the product to arrive? \
If this information is not found, output -1.""",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="""Extract any sentences about the value or price, \
and output them as a comma separated Python list.""",
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [0]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [0]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [0]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [0]:
prompt_with_context = PromptTemplate.from_template(review_template_2)
print(prompt_with_context.input_variables)
print(prompt_with_context.template)

llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)
print(llm_context_chain)

['format_instructions', 'text']
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: {text}

{format_instructions}

memory=None callbacks=None callback_manager=None verbose=False prompt=PromptTemplate(input_variables=['format_instructions', 'text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them 

In [0]:
# Call the LLM to translate to the style of the customer message
response = llm_context_chain.predict(
    text=customer_review, format_instructions=format_instructions
)
print(response)


True, two days,, $500, $800, awesome,,,,,


In [0]:
# You will get an error by running this cell of code 
# when the output response is not a dictionary
output_dict = output_parser.parse(response)
print(output_dict)